<a href="https://colab.research.google.com/github/mudogruer/LLMs/blob/main/LLama-7b-Zero-Shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U accelerate bitsandbytes bert-score evaluate

In [2]:
!pip install -q transformers["torch"]=="4.36.2"

In [3]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quanty_type = "nf4",
    bnb_4bit_use_double_quanty = True,
)


In [4]:
# from huggingface_hub import login
# login()

In [5]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"

from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left', trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code = True,

)

model.safetensors.index.json:   0%|          | 0.00/92.7k [00:00<?, ?B/s]

model-00001-of-00019.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00019.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

...

model-00017-of-00019.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00018-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00019-of-00019.safetensors:   0%|          | 0.00/4.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype = torch.bfloat16,
    device_map = "auto"

)


In [8]:
import torch

prompt = "<s>[INST]"+"What is the heaviest element?"+"[/INST]</s>:"
#prompt = "What is the heaviest element?"

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

sequence = pipe(
    prompt,
    do_sample = True,
    max_new_tokens = 100,
    temperature = 0.2,
    top_k = 2,
    top_p = 1.0,
)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [9]:
print(sequence[0]['generated_text'])

<s>[INST]What is the heaviest element?[/INST]</s>: The heaviest element in terms of atomic weight is Hassium (element 108, symbol Hs). However, if you're asking about the element with the greatest density, then it's Osmium (element 76, symbol Os). Please note that these facts are based on currently available information and are subject to change as new elements may be discovered or synthesized in the future.


In [10]:
sequence

[{'generated_text': "<s>[INST]What is the heaviest element?[/INST]</s>: The heaviest element in terms of atomic weight is Hassium (element 108, symbol Hs). However, if you're asking about the element with the greatest density, then it's Osmium (element 76, symbol Os). Please note that these facts are based on currently available information and are subject to change as new elements may be discovered or synthesized in the future."}]

In [11]:
prompt = "Instruct:"+"What is the gravity on mars?"+"\nOutput:"

sequence = pipe(
    prompt,
    do_sample = True,
    max_new_tokens = 100,
    temperature = 0.2,
    top_k = 2,
    top_p = 1.0,
)
print(sequence[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Instruct:What is the gravity on mars?
Output: The gravity on Mars is about 38% of the gravity on Earth.


In [12]:
sequence

[{'generated_text': 'Instruct:What is the gravity on mars?\nOutput: The gravity on Mars is about 38% of the gravity on Earth.'}]

In [13]:
def Sequence(promt):
    sequence = pipe(
    prompt,
    do_sample = True,
    max_new_tokens = 30,
    temperature = 0.1,
    top_k = 2,
    top_p = 1.0,
    pad_token_id=tokenizer.eos_token_id
    )
    return sequence

In [14]:
prompt = "<s>[INST]"+"What is the newest element?"+"[/INST]:</s>"

print(Sequence(prompt)[0]['generated_text'])

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


<s>[INST]What is the newest element?[/INST]:</s>The newest element on the periodic table is currently Nihonium (Nh, atomic number 113). It was officially added to the


In [15]:
Sequence(prompt)[0]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


{'generated_text': '<s>[INST]What is the newest element?[/INST]:</s>The newest element on the periodic table is currently Nihonium (Nh, atomic number 113). It was officially added to the'}

In [16]:
from datasets import load_dataset

dataset = load_dataset("sciq")

Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 11679
    })
    validation: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
})

In [18]:
dataset_validation = load_dataset('sciq',split='validation[:20%]') #first %20 of validation data

In [19]:
dataset_validation

Dataset({
    features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
    num_rows: 200
})

In [20]:
answers = []
for question in dataset_validation["question"]:
    prompt = "<s>[INST]"+question+"[/INST]</s>"
    output_sequences = Sequence(prompt)[0]['generated_text']
    question_and_answer = output_sequences.split("[/INST]")
    answer = question_and_answer[1]
    answers.append(answer)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

In [21]:
from bert_score import score

# Assuming `answers` is your list of generated answers and `actual_answers` is the list of reference answers
# Both should be lists of strings

# Calculate BertScore
P, R, F1 = score(answers, dataset_validation["correct_answer"], lang='en', verbose=True)

# `P`, `R`, and `F1` are returned as tensors, so you might want to convert them to Python lists or take their mean
bertscore_precision = P.mean().item()
bertscore_recall = R.mean().item()
bertscore_f1 = F1.mean().item()

print(f"Precision: {bertscore_precision:.3f}")
print(f"Recall: {bertscore_recall:.3f}")
print(f"F1: {bertscore_f1:.3f}")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/7 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 0.95 seconds, 209.86 sentences/sec
Precision: 0.786
Recall: 0.865
F1: 0.822


In [22]:
import pandas as pd
df = pd.DataFrame({"generated_answers":answers,"correct_answers":dataset_validation["correct_answer"]})

In [23]:
df.head(10)

,generated_answers,correct_answers
0,</s>The theory of evolution by natural selecti...,darwin
1,</s>Each specific polypeptide has a unique lin...,amino
2,</s>A frameshift mutation is a type of genetic...,nucleotides
3,</s>erland\nWetland\n\nWetland is the correct ...,wetland
4,</s>ield Worker> Surface waters are typically ...,the sun
5,"</s>`Arteries, veins, and capillaries are exam...",blood vessels
6,</s>The two general categories of biochemical ...,catabolic and anabolic
7,"</s>(data-old-href=""http://au.ipoll.com/172921...",volcanic ash
8,</s>The organ you're referring to is the human...,brain
9,"</s>rap (1) A length of rope, wire, or similar...",lariat


In [24]:
from evaluate import load
import numpy as np
bertscore = load("bertscore")
predictions = answers
references = dataset_validation["correct_answer"]
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
print("precision: ",round(np.mean(list(results["precision"])),5))
print("recall: ",round(np.mean(list(results["recall"])),5))
print("f1: ",round(np.mean(list(results["f1"])),5))

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

precision:  0.6295
recall:  0.78623
f1:  0.69795
